# Alabama Power Model Building Guide
## Building Machine Learning Models with AMI Data
---

## Overview
Over the last few days, we've talked at length about the tools and processes of data science and machine learning. Now it's time to bring it all together on a real business problem: errors in the labeling of transformer-customer pairings.

## The Data Science Process
There are two dominant workflows governing data mining and data processing in the world: OSEMN (Obtain, Scrub, Explore, Model, and iNterpret) and CRISP-DM (Cross Industry Standard Process for Data Mining). But they both hit the same main points:

1. Identify the problem
2. Find some data to that could conceivably solve the problem
3. Clean the Data
4. Explore the data
5. Model the data
6. Interpret the model that you've built
7. (Optional) Put the model into production

In this exercise, we will review the basic steps of the machine learning workflow. In practice, this will be an iterative process, where you return to previous steps to make improvements to your model. 

### Step 1: Identify the problem
* What problem are you trying to solve or understand? Remember that we generally work with data and build models for two reasons: inference and prediction. Either we are trying to understand a phenomenon (inference) or predict an unknown value from known quantities (prediction). Sometimes we do one en route to doing the other.
* Why does this problem matter? Who cares about it?
* What does success look like? Data science projects are seldom truly 'done.' They are usually 'done enough for now.'

In general, though not always, we can frame a machine learning problem as:
>Using `data` I will predict `target`. This is important because `rationale`, and the people/organizations who will benefit are `stakeholders` (optional).

`Data` are the data you are planning to use. What are the inputs to your model? Specifically, what features/columns will you be using? It is not sufficient to say, for example, "power data." We need to be specific. Which power data? Collected how and from what sources? What are the features? 

`Target` is what your model is trying to predict. Is this something that would realistically be unknown in a context where the features in `data` are unknown? Typically, this is a real number (regression) or a class label (classification). Sometimes, we don't have a class label and are trying to generate that from the data, which is an unsupervised learning problem.

`Rationale` is the business justification for what you're doing. Why would it be valuable to use this `data` to predict or generate this `target`? 

Strictly speaking, `stakeholder` consideration is optional, but it can be a valuable exercise that helps refine the purpose of the project. Who would find this model useful? How accurate does it need to be in order to be useful?

Other questions like "What is the relationship between A and B?" or "How much does C factor into D outcomes?" may be incidentally possible, but make sure you have that predictive framing first.

### Step 2: Find some data to that could conceivably solve the problem
The data in this case have already been provided. However, for future iterations of this problem, you might want to put together a 'wish list' of data (basically, the features, columns, and types if information) that you would like to have, that you think will solve the problem better.

* What data do we have?
* How many observations?
* How many columns/features?
* What are the features? What do they represent?

### Step 3: Clean the Data

Steps three and four can be a bit circular. At its core, cleaning data is about putting data into a format that your machine learning algorithms can handle. This includes, but is not limited to: converting strings into some kind of numerical representation, ensuring that timestamps are computer readable, making sure that numbers are in the correct format.

### Step 4: Explore the data

Data exploration, also called exploratory data analysis (EDA), is the process of asking initial questions of data. It's the analytics phase of data science and machine learning, as we try to figure out what is in the data at a surface level. It's also how we find new things that need cleaning: as we process the data further, we find new problems.

### Step 5: Design the Methodology

Finally, we're at the fun part. Once our data are clean (enough) and understood (enough), we can think about the machine learning approaches we can take. These approaches are informed by three things:
1. What is the problem we are trying to solve?
2. What kind of data do we have?
3. How much development time do we have?

To select the best approach, we need to have a good understanding of our data and the problem area we are working in. The problem of meter/transformer labeling has a small body of academic research around it. We can use these papers as a launch point to build out our methodology.

For our use case, data, and time frame, the approach proposed in, [Identification and Correction of Errors in Pairing AMI Meters and Transformers](https://www.osti.gov/biblio/1860605)(Blakey, 2021) will be the most appropriate for our use case. 

Blakey's process falls into two stages:

In the first stage, errors are flagged on a per-transformer basis.
1. Pearson correlation coefficients are calculated between each pair of customers (i.e., per voltmeter id).  
    1. (1) can be shortened by filtering for customers labeled as being on the same transformer
    2. Preprocessed to per-unit representations, and diffs between timesteps are used rather than raw value.
    3. Missingness handled with 4-day windows of data. Any customers with missing data in that period are discarded.
    4. EAch pair thus has multiple correlation coefficients, the mean of all of them is the corr for the pair.
2. Inspect Pairwise correlations for customers labeled on the same transformer.
3. Flag transformers containing customers below the treshold

In the second stage, new transformer groups are assigned.
1. Use pairwise linear regression to calculate MSE and reactance distance
    
Stage 1:
```python
For each transformer group
    if any CC < beta
        flag transformer
    else
        transformer group considered good, no further action
Sort transformer groups by lowest CC found
```

Stage 2:
```python
For each flagged transformer:
    for each customer labled on the flagged transformer:
        If the majority of CC with other customers in the group is < beta:
            Calculate pairwise regression with all other customers
               if and (MSE < mu) and ((x_1 + x_2) >Y)
                assign new transformer grouping to that set of customers
            Else:
                customer on transformer by itself
          Else:
            customer retains original transformer label
```

If you would like to learn different approaches, two other papers are also helpful:
* [Use of Smart-meter Data to Determine Distribution system Topology](https://www.scienceopen.com/document_file/aebbbd50-a30d-4af0-80da-e717ebe94e9a/API/JOE.2016.0033.pdf)
* [Project EPIC 2.14-Automatically Map Phasing Information.](https://www.researchgate.net/publication/330912460_Pacific_Gas_and_Electric_Company_Electric_Program_Investment_Charge_EPIC_Project_EPIC_214-Automatically_Map_Phasing_Information)

### 6. Evaluating the Model
Model building and evaluation are an iterative process. Once a model is built, we need to see how it did. Typically we evaluate its performance against a metric decided on as part of the problem description. The most important question is whether we improved on random chance (the baseline model) and, if so, by how much.

If we haven't improved on the baseline model, or haven't improved by enough, then we may need to start over again: re-evaluate our assumptions, or even the problem we're trying to solve. We can also try to tune the model by adjusting hyperparameters within our algorithm to see if that can juice our performance.

### 7. Interpreting the Model
Interpretation is about understanding what you and the computer have actually done. Once we're satisfied with the results (or we're out of time), turn your models predictions and scores and inputs into human language. For example:

>Using `data`, our model successfuly predicts `target` with a `score` accuracy. 